## Goals:
    1. Take in DF's from 'filtered' (using pandas)
    2. Read first row, get first item in species column
    3. Use species[1] to look up sequence in fasta file
    4. User first row, raw position # to get nucleotide position + n (length of motif)
    5. Put this nucleotide sequence into new column

TO-DO:
1. read in motif file and get motif length
2. make alignment file method for raw file
3. when reading file, need to account for negative/positive
4. make alignment file method --> need to account for spaces???

Account for negative string reading GGATTA
                                    CCTAAT
                                    
-bcd:
- GGATTA positive
- TAATCC negative

In [2]:
from Bio import SeqIO
from Bio import Seq
import pandas as pd
import numpy as np
import csv
import re
import os
from datascience import *

In [3]:
def seqToString(motif):
    """
    motif: a Seq that represents the motif 
    
    Returns the String representation of the motif
    """
    i = 0
    string = ""
    length = motif.__len__()
    while i < length:
        string += motif.__getitem__(i)
        i += 1
    return string

In [4]:
def getMotifLength(motif_key):
    f = open("../data/jaspar_fm/modified/" + motif_key, 'r')
    length = 0
    for line in f:
        a = line.split()
        length = len(a)
    return length

In [5]:
getMotifLength("MA0049.1_hb.jaspar")

10

In [6]:
def getNegative(pos_seq):
    """
    pos_seq = dna sequence in the positive direction reading from the file
    
    Returns the negative counterpart of the positive sequence.
    """
    dict = {"A":'T','T':'A','G':'C','C':'G','-':'-'}
    negative = ""
    last_index = len(pos_seq) - 1
    while last_index > -1:
        negative += dict[pos_seq[last_index].upper()]
        last_index -= 1
    return negative

In [7]:
def getRawMotifStrings(csv_file):
    """
    csv_file: a csv file that contains the 'filtered' data frame
    
    Creates a reader representing the data frame and iterates through each line retrieving the species ID and key.
    Parses through the raw file to find the motif sequence.
    Returns an data frame of the motifs.
    """
    f = open("../data/output/bcd/" + csv_file, 'r')
    data = csv.reader(f)
    line0 = next(data)
    motifs = []
    for line in data:
        species = line[2]
        direction = line[4]
        raw_index = int(float(line[3]))
        key = species.split('|')[0]
        record_dict = SeqIO.to_dict(SeqIO.parse("../data/raw/outlier_rm_with_length_" + key + ".fa", "fasta"))
        sequence = record_dict[species]
        motif_key = line[6]
        motif_length = getMotifLength(motif_key);
        motif = sequence[raw_index:raw_index + motif_length]
        string_motif = seqToString(motif)
        if direction == "negative":
            string_motif = getNegative(string_motif)
        motifs += [string_motif]
    f.close()
    motif_df = pd.DataFrame({'motif sequences': motifs})
    return motifs

In [8]:
def getAlignMotifStrings(csv_file):
    f = open("../data/output/bcd/" + csv_file, 'r')
    data = csv.reader(f)
    line0 = next(data)
    motifs = []
    for line in data:
        species = line[2]
        direction = line[4]
        align_index = int(float(line[5]))
        key = species.split('|')[0]
        record_dict = SeqIO.to_dict(SeqIO.parse("../data/alignments/align_outlier_rm_with_length_" + key + ".fa", "fasta"))
        sequence = record_dict[species]
        motif_key = line[6]
        motif_length = getMotifLength(motif_key);
        motif = sequence[align_index:align_index + motif_length]
        string_motif = seqToString(motif)
        if direction == "negative":
            string_motif = getNegative(string_motif)
        motifs += [string_motif]
    f.close()
    motif_df = pd.DataFrame({'motif sequences': motifs})
    return motifs

In [9]:
motif = getAlignMotifStrings("occurance_align_outlier_rm_with_length_VT6436.fa.csv")
df = pd.DataFrame()
df["VT6436.fa.csv"] = motif
# print(df)
motif2 = getAlignMotifStrings("occurance_align_outlier_rm_with_length_VT6705.fa.csv")
df1 = pd.DataFrame()
df1["VT6705.fa.csv"] = motif2
# print(df1)
df = pd.concat([df,df1], ignore_index=False, axis = 1)
#print(df)

FileNotFoundError: [Errno 2] No such file or directory: '../data/output/bcd/occurance_align_outlier_rm_with_length_VT6436.fa.csv'

In [26]:
df = pd.DataFrame()
if df.empty:
    print("empty")

empty


In [27]:
def alignmentsDataFrame():
    csvList = os.listdir("../data/output/bcd/")
    df = pd.DataFrame()
    for file in csvList:
        motif = getAlignMotifStrings(file)
        df1 = pd.DataFrame()
        df1[file[39:]] = motif
        if df.empty:
            df[file[39:]] = motif
        else:
            df = pd.concat([df,df1], ignore_index=False, axis=1)
    new_csv = df.to_csv("../data/output/2.motif_extraction_output/align/bcd_motif.fa.csv")
    print(df)

In [28]:
alignmentsDataFrame()

    VT11048.fa.csv VT16679.fa.csv VT19895.fa.csv VT40548.fa.csv  \
0           TAATCC         TTATCC         TAATCC         TAATCC   
1           TAATCC         -----C         TT-ATC         TTATCC   
2              NaN         TAATCC         TAATCC         TTATCC   
3              NaN         TTATCC         TT-ATC         TAATCC   
4              NaN         T-----         TAATCC         TTATCC   
5              NaN         -----C         TAATCC         TAATCC   
6              NaN         TAATCC         TAATCC         TAATCC   
7              NaN         TTATCC         TAATCC         TTATCC   
8              NaN         TTATCC         A--TCC         TAATCC   
9              NaN         TTATCC         TTATCC         TAATCC   
10             NaN         -----C         TAATCC         TAATCC   
11             NaN         TAATCC         A--TCC         TAATCC   
12             NaN         TTATCC         TAATCC         TTA---   
13             NaN         TTATCC         TAATCC         TAATC

In [29]:
def csvToDataFrame(csv_file):
    df = pd.read_csv("../data/output/bcd/" + csv_file);
    return df

In [30]:
def csvToArray(csv_file):
    f = open(csv_file, 'r')
    data = csv.reader(f)
    labels = next(data)
    labels.append('motif sequence')
    array = []
    array.append(labels)
    
    for row in data:
        array.append(row)
    return array

In [35]:
def addRawMotifStrings(csv_file):
    """
    csv_file: a csv file that contains the 'filtered' data frame
    
    Appends the DF of motif sequences as a column onto the filtered DF.
    """
    raw_motif_seq = getRawMotifStrings(csv_file)
    df = csvToDataFrame(csv_file)
    df['raw motif sequences'] = raw_motif_seq
    new_csv = df.to_csv("../data/output/2.motif_extraction_output/raw/motif+" + csv_file)
    return df

In [36]:
d = addRawMotifStrings("occurance_align_outlier_rm_with_length_VT11048.fa.csv")

In [37]:
d

,Unnamed: 0,score,species,raw_position,strand,align_position,motif,raw motif sequences
0,0,11.612828,VT11048|0|MEMB005D|-|287,39,negative,60,MA0212.1_bcd.jaspar,TAATCC
1,1,11.612828,VT11048|0|MEMB006A|+|286,38,negative,60,MA0212.1_bcd.jaspar,TAATCC


In [179]:
csvList = os.listdir("../data/output/bcd/")
for file in csvList:
    addRawMotifStrings(file)

In [111]:
species = [11048, 16679, 19895, 40548, 43692, 44110, 48156, 50550, 59000, 60074, 61209, 6436, 6705, 7859, 8646]
member = ['Dkik', 'MEMB002A', 'MEMB002B', 'MEMB002C', 'MEMB002D', 'MEMB002E', 'MEMB002F', 'MEMB003A', 'MEMB003B', 'MEMB003C', 'MEMB003D',
          'MEMB003E', 'MEMB003F', 'MEMB004A', 'MEMB004B', 'MEMB004E', 'MEMB005D', 'MEMB006B', 'MEMB006C', 'MEMB007A', 'MEMB007B', 
          'MEMB007C', 'MEMB007D', 'MEMB008C']

In [21]:
def getAlignPositions(species):
    table = Table.read_table("../data/output/map_motif_bcd_with_threshold/occurance_align_outlier_rm_with_length_VT" + str(species) + ".fa.csv")
    align = table.group('align_position').drop('count').relabel('align_position', 'align_' +  str(species)).column('align_' +  str(species))
    return align   


In [29]:
a = getAlignPositions(11048)
a

array([60])

In [23]:
spec_align_dict = {}
for s in species:
    a = getAlignPositions(s)
    spec_align_dict[s] = a
print(spec_align_dict)

{11048: array([60]), 16679: array([ 502, 1200, 1202, 1316, 1481, 1716, 1717, 2073, 2114, 2202, 2606,
       2612, 2618]), 19895: array([  93,  518,  611, 2385, 2444, 3390]), 40548: array([ 619,  672,  764,  867,  960, 1077, 1278, 1736, 1905, 1963, 1967,
       2020, 2649, 2703, 3011, 3028, 3234, 3243, 3244, 3269, 3306, 3307,
       3318, 3327, 3465, 3493, 3902]), 43692: array([ 600,  677,  685,  744, 1308, 1750, 1813, 2554, 2622, 2915, 3098,
       3295]), 44110: array([  41,  581,  696,  715,  753,  854,  995, 1013, 1049, 1716, 1745,
       1943, 1955, 2071]), 48156: array([153]), 50550: array([ 325,  345,  527,  589,  617,  906, 1092, 1111, 1114, 1124, 1152,
       1158, 1183, 1593, 1992, 2923, 3132, 3333]), 59000: array([ 457,  647,  934, 1881, 3177, 3330, 3339, 3806, 3841, 3975, 4043,
       4739, 5411, 5432, 5468, 5545, 5626, 5991, 6378]), 60074: array([ 44, 575, 866, 940]), 61209: array([ 579, 1064, 1077, 2099, 2951, 3341, 3360, 3575, 3680, 3689, 5942,
       6074, 7345, 7628]), 

In [109]:
def getRawPositions(species, length):
    table = Table.read_table("../data/output/map_motif_bcd_no_threshold/VT" + str(species) + ".fa.csv")
    spec = Table()
    raw_position = []
    a = spec_align_dict[species]
    print(a)
    for pos in a:
        print(table)
        raw = table.where('align_position', are.equal_to(pos)).where('strand', are.equal_to('positive'))
        raw = raw.drop('strand', 'Unnamed: 0', 'motif').relabel('raw_position', str(pos))
        print("raw", raw)
        arr = raw.column(str(pos))
        print(arr)
        spec = Table().with_columns('raw', arr, 'orig_species', raw.column('species'), 'score', raw.column('score'), 'align_position', raw.column('align_position'))
        spec.show()
        record_dict = SeqIO.to_dict(SeqIO.parse("../data/raw/outlier_rm_with_length_VT" + str(species) + ".fa", "fasta"))
        species_dict = dict(zip(spec.column(1), spec.column(0)))
        print(species_dict)
        seq = []
        before = []
        after = []
        all_pos = Table()
        raw_arr = spec.column('raw')
        species_arr = spec.column('orig_species')
        score_arr = spec.column('score')
        align_arr = spec.column('align_position')
        for key in species_dict: 
            print(key)
            sequence = record_dict[key]
            seq.append(seqToString(sequence[species_dict[key]:species_dict[key] + 6]))
            before.append(seqToString(sequence[species_dict[key] - length:species_dict[key]]))
            after.append(seqToString(sequence[species_dict[key] + 6:species_dict[key] + 6 + length]))
            print(len(seq), len(before), len(after))
        spec_updated = spec.with_columns('sequence', seq, str(length) + '_before', before, str(length) + '_after', after)
        spec_df = spec_updated.to_df()
        print(spec_df)
        if not os.path.isdir("../data/output/2.motif_extraction_output/raw_seq_per_align/" + str(species)):
            os.mkdir("../data/output/2.motif_extraction_output/raw_seq_per_align/" + str(species))
        spec_df.to_csv("../data/output/2.motif_extraction_output/raw_seq_per_align/" + str(species) + "/" + str(species) + "_" + str(pos) + "_raw.fa.csv")

In [112]:
for spec in species:
    getRawPositions(spec, 5)


[60]
Unnamed: 0 | score    | species                  | raw_position | strand   | align_position | motif
2778       | -6.14023 | VT11048|0|MEMB002A|+|284 | 0            | positive | 21             | bcd_FlyReg.fm
2779       | 1.09259  | VT11048|0|MEMB002A|+|284 | 0            | negative | 21             | bcd_FlyReg.fm
2780       | -6.14023 | VT11048|0|MEMB002A|+|284 | 1            | positive | 22             | bcd_FlyReg.fm
2781       | 1.09259  | VT11048|0|MEMB002A|+|284 | 1            | negative | 22             | bcd_FlyReg.fm
2782       | -1.8923  | VT11048|0|MEMB002A|+|284 | 2            | positive | 23             | bcd_FlyReg.fm
2783       | 3.35563  | VT11048|0|MEMB002A|+|284 | 2            | negative | 23             | bcd_FlyReg.fm
2784       | -4.14023 | VT11048|0|MEMB002A|+|284 | 3            | positive | 24             | bcd_FlyReg.fm
2785       | -3.30734 | VT11048|0|MEMB002A|+|284 | 3            | negative | 24             | bcd_FlyReg.fm
2786       | 2.35563  | VT11048

raw,orig_species,score,align_position
39,VT11048|0|MEMB002A|+|284,-5.20163,60
39,VT11048|0|MEMB002B|-|284,-5.20163,60
39,VT11048|0|MEMB002C|-|284,-5.20163,60
35,VT11048|0|MEMB002D|+|269,-4.20163,60
40,VT11048|0|MEMB002E|+|288,0.0078255,60
36,VT11048|0|MEMB002F|+|278,-5.20163,60
39,VT11048|0|MEMB003A|+|278,0.0078255,60
43,VT11048|0|MEMB003B|+|289,-5.20163,60
39,VT11048|0|MEMB003C|-|284,-5.20163,60
35,VT11048|0|MEMB003D|+|274,0.0078255,60


{'VT11048|0|MEMB002A|+|284': 39, 'VT11048|0|MEMB002B|-|284': 39, 'VT11048|0|MEMB002C|-|284': 39, 'VT11048|0|MEMB002D|+|269': 35, 'VT11048|0|MEMB002E|+|288': 40, 'VT11048|0|MEMB002F|+|278': 36, 'VT11048|0|MEMB003A|+|278': 39, 'VT11048|0|MEMB003B|+|289': 43, 'VT11048|0|MEMB003C|-|284': 39, 'VT11048|0|MEMB003D|+|274': 35, 'VT11048|0|MEMB003F|-|285': 39, 'VT11048|0|MEMB004A|+|284': 39, 'VT11048|0|MEMB004B|+|285': 39, 'VT11048|0|MEMB004E|-|285': 39, 'VT11048|0|MEMB005B|+|298': 60, 'VT11048|0|MEMB005D|-|287': 39, 'VT11048|0|MEMB006A|+|286': 38, 'VT11048|0|MEMB006B|-|275': 38, 'VT11048|0|MEMB006C|-|285': 39, 'VT11048|0|MEMB007B|-|284': 39, 'VT11048|0|MEMB007C|+|283': 38, 'VT11048|0|MEMB007D|-|274': 38, 'VT11048|0|MEMB008C|-|285': 39, 'VT11048|0|dkik|-|290': 39}
VT11048|0|MEMB002A|+|284
1 1 1
VT11048|0|MEMB002B|-|284
2 2 2
VT11048|0|MEMB002C|-|284
3 3 3
VT11048|0|MEMB002D|+|269
4 4 4
VT11048|0|MEMB002E|+|288
5 5 5
VT11048|0|MEMB002F|+|278
6 6 6
VT11048|0|MEMB003A|+|278
7 7 7
VT11048|0|MEMB003B

raw,orig_species,score,align_position
398,VT16679|0|MEMB002A|-|1805,-6.55526,502
370,VT16679|0|MEMB002B|+|1762,-6.55526,502
398,VT16679|0|MEMB002C|+|1908,-6.55526,502
423,VT16679|0|MEMB002D|-|1976,-6.40326,502
452,VT16679|0|MEMB002E|-|1989,-6.40326,502
401,VT16679|0|MEMB002F|-|1741,-6.40326,502
411,VT16679|0|MEMB003A|-|1764,-6.40326,502
443,VT16679|0|MEMB003B|-|1981,-6.40326,502
398,VT16679|0|MEMB003C|-|1817,-6.55526,502
411,VT16679|0|MEMB003D|+|1762,-6.40326,502


{'VT16679|0|MEMB002A|-|1805': 398, 'VT16679|0|MEMB002B|+|1762': 370, 'VT16679|0|MEMB002C|+|1908': 398, 'VT16679|0|MEMB002D|-|1976': 423, 'VT16679|0|MEMB002E|-|1989': 452, 'VT16679|0|MEMB002F|-|1741': 401, 'VT16679|0|MEMB003A|-|1764': 411, 'VT16679|0|MEMB003B|-|1981': 443, 'VT16679|0|MEMB003C|-|1817': 398, 'VT16679|0|MEMB003D|+|1762': 411, 'VT16679|0|MEMB003F|+|1932': 431, 'VT16679|0|MEMB004A|+|1736': 370, 'VT16679|0|MEMB004B|+|1798': 404, 'VT16679|0|MEMB004E|-|1923': 419, 'VT16679|0|MEMB005B|-|1889': 425, 'VT16679|0|MEMB005D|+|1902': 431, 'VT16679|0|MEMB006A|+|1883': 399, 'VT16679|0|MEMB006B|+|2261': 424, 'VT16679|0|MEMB006C|+|1921': 432, 'VT16679|0|MEMB007B|+|1813': 403, 'VT16679|0|MEMB007C|-|1948': 427, 'VT16679|0|MEMB007D|+|2039': 420, 'VT16679|0|MEMB008C|-|1904': 406, 'VT16679|0|dkik|+|1769': 411}
VT16679|0|MEMB002A|-|1805
1 1 1
VT16679|0|MEMB002B|+|1762
2 2 2
VT16679|0|MEMB002C|+|1908
3 3 3
VT16679|0|MEMB002D|-|1976
4 4 4
VT16679|0|MEMB002E|-|1989
5 5 5
VT16679|0|MEMB002F|-|1741
6

...

{'VT8646|0|MEMB002A|+|3062': 2411, 'VT8646|0|MEMB002B|+|2935': 2276, 'VT8646|0|MEMB002C|-|2975': 2318, 'VT8646|0|MEMB002D|+|3516': 2656, 'VT8646|0|MEMB002E|-|4143': 3295, 'VT8646|0|MEMB002F|+|3753': 2953, 'VT8646|0|MEMB003A|+|2540': 1902, 'VT8646|0|MEMB003B|+|2425': 1733, 'VT8646|0|MEMB003D|+|2591': 1909, 'VT8646|0|MEMB003F|+|2578': 1910, 'VT8646|0|MEMB004A|-|2919': 2261, 'VT8646|0|MEMB004B|+|2508': 1859, 'VT8646|0|MEMB004E|+|1999': 1336, 'VT8646|0|MEMB005B|-|2558': 1884, 'VT8646|0|MEMB005D|-|3092': 2393, 'VT8646|0|MEMB006B|-|2620': 1952, 'VT8646|0|MEMB006C|-|2565': 1906, 'VT8646|0|MEMB007C|+|2375': 1691, 'VT8646|0|MEMB007D|+|3583': 2909, 'VT8646|0|MEMB008C|+|2558': 1902, 'VT8646|0|dkik|+|2572': 1921}
VT8646|0|MEMB002A|+|3062
1 1 1
VT8646|0|MEMB002B|+|2935
2 2 2
VT8646|0|MEMB002C|-|2975
3 3 3
VT8646|0|MEMB002D|+|3516
4 4 4
VT8646|0|MEMB002E|-|4143
5 5 5
VT8646|0|MEMB002F|+|3753
6 6 6
VT8646|0|MEMB003A|+|2540
7 7 7
VT8646|0|MEMB003B|+|2425
8 8 8
VT8646|0|MEMB003D|+|2591
9 9 9
VT8646|0|M

raw,orig_species,score,align_position
2629,VT8646|0|MEMB002A|+|3062,-3.30734,5543
2464,VT8646|0|MEMB002B|+|2935,-4.20163,5543
2536,VT8646|0|MEMB002C|-|2975,-6.55526,5543
3049,VT8646|0|MEMB002D|+|3516,-3.30734,5543
3657,VT8646|0|MEMB002E|-|4143,-6.55526,5543
3296,VT8646|0|MEMB002F|+|3753,-4.20163,5543
2093,VT8646|0|MEMB003A|+|2540,-4.20163,5543
1948,VT8646|0|MEMB003B|+|2425,-4.20163,5543
2488,VT8646|0|MEMB003C|+|2927,-6.55526,5543
2123,VT8646|0|MEMB003D|+|2591,-4.20163,5543


{'VT8646|0|MEMB002A|+|3062': 2629, 'VT8646|0|MEMB002B|+|2935': 2464, 'VT8646|0|MEMB002C|-|2975': 2536, 'VT8646|0|MEMB002D|+|3516': 3049, 'VT8646|0|MEMB002E|-|4143': 3657, 'VT8646|0|MEMB002F|+|3753': 3296, 'VT8646|0|MEMB003A|+|2540': 2093, 'VT8646|0|MEMB003B|+|2425': 1948, 'VT8646|0|MEMB003C|+|2927': 2488, 'VT8646|0|MEMB003D|+|2591': 2123, 'VT8646|0|MEMB003F|+|2578': 2110, 'VT8646|0|MEMB004A|-|2919': 2449, 'VT8646|0|MEMB004B|+|2508': 2051, 'VT8646|0|MEMB004E|+|1999': 1532, 'VT8646|0|MEMB005B|-|2558': 2090, 'VT8646|0|MEMB005D|-|3092': 2607, 'VT8646|0|MEMB006A|-|3077': 2592, 'VT8646|0|MEMB006B|-|2620': 2141, 'VT8646|0|MEMB006C|-|2565': 2100, 'VT8646|0|MEMB007B|-|2865': 2426, 'VT8646|0|MEMB007C|+|2375': 1909, 'VT8646|0|MEMB008C|+|2558': 2091, 'VT8646|0|dkik|+|2572': 2112}
VT8646|0|MEMB002A|+|3062
1 1 1
VT8646|0|MEMB002B|+|2935
2 2 2
VT8646|0|MEMB002C|-|2975
3 3 3
VT8646|0|MEMB002D|+|3516
4 4 4
VT8646|0|MEMB002E|-|4143
5 5 5
VT8646|0|MEMB002F|+|3753
6 6 6
VT8646|0|MEMB003A|+|2540
7 7 7
VT86

raw,orig_species,score,align_position
2679,VT8646|0|MEMB002A|+|3062,-4.20163,5599
2514,VT8646|0|MEMB002B|+|2935,-4.20163,5599
2586,VT8646|0|MEMB002C|-|2975,-4.20163,5599
3093,VT8646|0|MEMB002D|+|3516,-4.20163,5599
3702,VT8646|0|MEMB002E|-|4143,-4.20163,5599
3342,VT8646|0|MEMB002F|+|3753,-4.20163,5599
2138,VT8646|0|MEMB003A|+|2540,-4.20163,5599
1997,VT8646|0|MEMB003B|+|2425,-4.20163,5599
2538,VT8646|0|MEMB003C|+|2927,-4.20163,5599
2168,VT8646|0|MEMB003D|+|2591,-4.20163,5599


{'VT8646|0|MEMB002A|+|3062': 2679, 'VT8646|0|MEMB002B|+|2935': 2514, 'VT8646|0|MEMB002C|-|2975': 2586, 'VT8646|0|MEMB002D|+|3516': 3093, 'VT8646|0|MEMB002E|-|4143': 3702, 'VT8646|0|MEMB002F|+|3753': 3342, 'VT8646|0|MEMB003A|+|2540': 2138, 'VT8646|0|MEMB003B|+|2425': 1997, 'VT8646|0|MEMB003C|+|2927': 2538, 'VT8646|0|MEMB003D|+|2591': 2168, 'VT8646|0|MEMB003F|+|2578': 2156, 'VT8646|0|MEMB004A|-|2919': 2498, 'VT8646|0|MEMB004B|+|2508': 2096, 'VT8646|0|MEMB004E|+|1999': 1577, 'VT8646|0|MEMB005B|-|2558': 2135, 'VT8646|0|MEMB005D|-|3092': 2663, 'VT8646|0|MEMB006A|-|3077': 2648, 'VT8646|0|MEMB006B|-|2620': 2186, 'VT8646|0|MEMB006C|-|2565': 2150, 'VT8646|0|MEMB007B|-|2865': 2476, 'VT8646|0|MEMB007C|+|2375': 1956, 'VT8646|0|MEMB007D|+|3583': 3158, 'VT8646|0|MEMB008C|+|2558': 2136, 'VT8646|0|dkik|+|2572': 2147}
VT8646|0|MEMB002A|+|3062
1 1 1
VT8646|0|MEMB002B|+|2935
2 2 2
VT8646|0|MEMB002C|-|2975
3 3 3
VT8646|0|MEMB002D|+|3516
4 4 4
VT8646|0|MEMB002E|-|4143
5 5 5
VT8646|0|MEMB002F|+|3753
6 6 6
V

In [25]:
#-verify whether or not the motif occurs --> the species that are in the original table
#-also include the score
#-from 
#-keep going w positive sequence make length a variable

In [53]:
table = Table.read_table("../data/output/map_motif_bcd_no_threshold/VT11048.fa.csv")
spec = Table().with_column('species', ['MEMB002A', 'MEMB002B', 'MEMB002C', 'MEMB002D', 'MEMB002E', 'MEMB002F', 'MEMB003A', 'MEMB003B', 'MEMB003C', 'MEMB003D', 'MEMB003E', 'MEMB003F', 'MEMB004A', 'MEMB004B', 'MEMB004E', 'MEMB005D', 'MEMB006B', 'MEMB006C', 'MEMB007A', 'MEMB007B', 'MEMB007C', 'MEMB007D', 'MEMB008C','dkik'])
raw_position = []
for pos in a:
    print(table)
    raw = table.where('align_position', are.equal_to(pos)).where('strand', are.equal_to('positive'))
    raw = raw.drop('strand', 'Unnamed: 0', 'motif').relabel('raw_position', str(pos))
    print(raw)
    arr = raw.column(str(pos))
    print(arr)
    spec = spec.with_columns('raw', arr, 'orig_species', raw.column('species'), 'score', raw.column('score'), 'align_position', raw.column('align_position'))
    spec.show()


Unnamed: 0 | score    | species                  | raw_position | strand   | align_position | motif
2778       | -6.14023 | VT11048|0|MEMB002A|+|284 | 0            | positive | 21             | bcd_FlyReg.fm
2779       | 1.09259  | VT11048|0|MEMB002A|+|284 | 0            | negative | 21             | bcd_FlyReg.fm
2780       | -6.14023 | VT11048|0|MEMB002A|+|284 | 1            | positive | 22             | bcd_FlyReg.fm
2781       | 1.09259  | VT11048|0|MEMB002A|+|284 | 1            | negative | 22             | bcd_FlyReg.fm
2782       | -1.8923  | VT11048|0|MEMB002A|+|284 | 2            | positive | 23             | bcd_FlyReg.fm
2783       | 3.35563  | VT11048|0|MEMB002A|+|284 | 2            | negative | 23             | bcd_FlyReg.fm
2784       | -4.14023 | VT11048|0|MEMB002A|+|284 | 3            | positive | 24             | bcd_FlyReg.fm
2785       | -3.30734 | VT11048|0|MEMB002A|+|284 | 3            | negative | 24             | bcd_FlyReg.fm
2786       | 2.35563  | VT11048|0|ME

species,raw,orig_species,score,align_position
MEMB002A,39,VT11048|0|MEMB002A|+|284,-5.20163,60
MEMB002B,39,VT11048|0|MEMB002B|-|284,-5.20163,60
MEMB002C,39,VT11048|0|MEMB002C|-|284,-5.20163,60
MEMB002D,35,VT11048|0|MEMB002D|+|269,-4.20163,60
MEMB002E,40,VT11048|0|MEMB002E|+|288,0.0078255,60
MEMB002F,36,VT11048|0|MEMB002F|+|278,-5.20163,60
MEMB003A,39,VT11048|0|MEMB003A|+|278,0.0078255,60
MEMB003B,43,VT11048|0|MEMB003B|+|289,-5.20163,60
MEMB003C,39,VT11048|0|MEMB003C|-|284,-5.20163,60
MEMB003D,35,VT11048|0|MEMB003D|+|274,0.0078255,60


{'VT11048|0|MEMB002A|+|284': 39, 'VT11048|0|MEMB002B|-|284': 39, 'VT11048|0|MEMB002C|-|284': 39, 'VT11048|0|MEMB002D|+|269': 35, 'VT11048|0|MEMB002E|+|288': 40, 'VT11048|0|MEMB002F|+|278': 36, 'VT11048|0|MEMB003A|+|278': 39, 'VT11048|0|MEMB003B|+|289': 43, 'VT11048|0|MEMB003C|-|284': 39, 'VT11048|0|MEMB003D|+|274': 35, 'VT11048|0|MEMB003F|-|285': 39, 'VT11048|0|MEMB004A|+|284': 39, 'VT11048|0|MEMB004B|+|285': 39, 'VT11048|0|MEMB004E|-|285': 39, 'VT11048|0|MEMB005B|+|298': 60, 'VT11048|0|MEMB005D|-|287': 39, 'VT11048|0|MEMB006A|+|286': 38, 'VT11048|0|MEMB006B|-|275': 38, 'VT11048|0|MEMB006C|-|285': 39, 'VT11048|0|MEMB007B|-|284': 39, 'VT11048|0|MEMB007C|+|283': 38, 'VT11048|0|MEMB007D|-|274': 38, 'VT11048|0|MEMB008C|-|285': 39, 'VT11048|0|dkik|-|290': 39}
     species  raw              orig_species     score  align_position
0   MEMB002A   39  VT11048|0|MEMB002A|+|284 -5.201628              60
1   MEMB002B   39  VT11048|0|MEMB002B|-|284 -5.201628              60
2   MEMB002C   39  VT1104

In [41]:
record_dict = SeqIO.to_dict(SeqIO.parse("../data/raw/outlier_rm_with_length_VT11048.fa", "fasta"))
species_dict = dict(zip(spec.column(2), spec.column(1)))
print(species_dict)
seq = []
five_before = []
five_after = []
for key in species_dict:
    sequence = record_dict[key]
    seq.append(seqToString(sequence[species_dict[key]:species_dict[key] + 6]))
    five_before.append(seqToString(sequence[species_dict[key] - 5:species_dict[key]]))
    five_after.append(seqToString(sequence[species_dict[key] + 6:species_dict[key] + 11]))
spec.with_columns('sequence', seq, 'five_before', five_before, 'five_after', five_after)
spec_df = spec.to_df()
print(spec_df)
spec_df.to_csv("../data/output/2.motif_extraction_output/raw_seq_per_align/11048_raw.fa.csv")


{'VT11048|0|MEMB002A|+|284': 39, 'VT11048|0|MEMB002B|-|284': 39, 'VT11048|0|MEMB002C|-|284': 39, 'VT11048|0|MEMB002D|+|269': 35, 'VT11048|0|MEMB002E|+|288': 40, 'VT11048|0|MEMB002F|+|278': 36, 'VT11048|0|MEMB003A|+|278': 39, 'VT11048|0|MEMB003B|+|289': 43, 'VT11048|0|MEMB003C|-|284': 39, 'VT11048|0|MEMB003D|+|274': 35, 'VT11048|0|MEMB003F|-|285': 39, 'VT11048|0|MEMB004A|+|284': 39, 'VT11048|0|MEMB004B|+|285': 39, 'VT11048|0|MEMB004E|-|285': 39, 'VT11048|0|MEMB005B|+|298': 60, 'VT11048|0|MEMB005D|-|287': 39, 'VT11048|0|MEMB006A|+|286': 38, 'VT11048|0|MEMB006B|-|275': 38, 'VT11048|0|MEMB006C|-|285': 39, 'VT11048|0|MEMB007B|-|284': 39, 'VT11048|0|MEMB007C|+|283': 38, 'VT11048|0|MEMB007D|-|274': 38, 'VT11048|0|MEMB008C|-|285': 39, 'VT11048|0|dkik|-|290': 39}
     species  raw              orig_species     score  align_position
0   MEMB002A   39  VT11048|0|MEMB002A|+|284 -5.201628              60
1   MEMB002B   39  VT11048|0|MEMB002B|-|284 -5.201628              60
2   MEMB002C   39  VT1104

In [150]:
def combineCSV(species):
    final_table = Table()
    raw = np.empty(0)
    orig_species = np.empty(0)
    score = np.empty(0)
    align_position = np.empty(0)
    sequence = np.empty(0)
    before = np.empty(0)
    after = np.empty(0)
    for file in os.listdir("../data/output/2.motif_extraction_output/raw_seq_per_align/" + str(species)):
        table = Table().read_table("../data/output/2.motif_extraction_output/raw_seq_per_align/" + str(species) + "/" + file)
        raw = np.append(raw, table.column('raw'))
        orig_species = np.append(orig_species, table.column('orig_species'))
        score = np.append(score, table.column('score'))
        align_position = np.append(align_position, table.column('align_position'))
        sequence = np.append(sequence, table.column('sequence'))
        before = np.append(before, table.column(5))
        after = np.append(after, table.column(6))
    final_table = final_table.with_columns('raw', raw, 'orig_species', orig_species, 'score', score, 'align_position', align_position, 'sequence', sequence, 'before', before, 'after', after)
    final_table.to_csv("../data/output/2.motif_extraction_output/raw_seq_per_align/" + str(species) + "/" + str(species) + "_final_raw.fa.csv")

In [153]:
for s in species:
    combineCSV(s)

In [149]:
x = np.empty(0)
x = np.append(x, make_array(12, 34, 23, 34))
x

array([12., 34., 23., 34.])